# 01 — Linear Regression (Hands-on)

Objectives:
- Fit a linear regression model and evaluate performance (R^2, MAE, RMSE)
- Validate assumptions: linearity, independence, homoscedasticity, normality of residuals, and low multicollinearity
- Use diagnostic plots (residual vs. fitted, QQ plot)
- Apply statistical tests (Breusch–Pagan for heteroskedasticity) and VIF for multicollinearity
- Practice feature scaling/engineering and handling outliers

Assumptions to remember:
- Linearity: relationship between features and target is linear
- Independence: observations are independent
- Homoscedasticity: residual variance is constant
- Normality of Errors: residuals are approximately normal (for CI/PI validity)
- Low Multicollinearity: predictors are not highly correlated

Cautions/Data Prep:
- Outliers can distort fit; consider removal/transformations
- Scale features if using regularization or polynomial features
- Handle missing data; linear regression doesn’t natively handle NaNs
- Always check diagnostics after fitting


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_regression, fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

sns.set(style='whitegrid', context='notebook')
np.random.seed(42)


## 1) Load a simple synthetic dataset
We start with a clean linear signal with noise. This is fast and ideal on CPU. Optionally, try California Housing later (slower but realistic).

In [ ]:
X, y = make_regression(n_samples=1000, n_features=5, n_informative=5, noise=15.0, random_state=42)
feature_names = [f"x{i+1}" for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=feature_names)
df['y'] = y
df.head()

Train/validation split, then fit a simple LinearRegression (no regularization).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[feature_names], df['y'], test_size=0.2, random_state=42)
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred = linreg.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2, mae, rmse

Inspect coefficients and intercept. Scale matters when interpreting coefficients; with standardized features, magnitudes are comparable.

In [ ]:
coef_df = pd.DataFrame({'feature': feature_names, 'coefficient': linreg.coef_}).sort_values('coefficient')
display(pd.Series({'intercept': linreg.intercept_}))
coef_df

## 2) Residual diagnostics
- Residual vs. predicted: look for randomness (no clear pattern)
- Histogram/QQ plot: check approximate normality
- Homoscedasticity test (Breusch–Pagan)


In [ ]:
residuals = y_test - y_pred

fig, axes = plt.subplots(1, 3, figsize=(16, 4))
axes[0].scatter(y_pred, residuals, alpha=0.6)
axes[0].axhline(0, color='red', linestyle='--')
axes[0].set_title('Residuals vs Predicted')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Residuals')

sns.histplot(residuals, kde=True, ax=axes[1])
axes[1].set_title('Residuals Distribution')

sm.ProbPlot(residuals, fit=True).qqplot(line='45', ax=axes[2])
axes[2].set_title('QQ Plot of Residuals')
plt.tight_layout()
plt.show()

# Normality tests (indicative)
jb_stat, jb_p = stats.jarque_bera(residuals)
sw_stat, sw_p = stats.shapiro(residuals)
print(f"Jarque-Bera p-value: {jb_p:.4f}")
print(f"Shapiro-Wilk p-value: {sw_p:.4f}")


Breusch–Pagan test for heteroskedasticity. H0: constant variance (homoscedastic). Small p-value suggests heteroskedasticity.

In [ ]:
X_test_const = sm.add_constant(X_test)
ols = sm.OLS(y_test, X_test_const).fit()
bp_test = het_breuschpagan(ols.resid, X_test_const)
labels = ['LM stat', 'LM p-value', 'F stat', 'F p-value']
pd.Series(bp_test, index=labels)

## 3) Multicollinearity via VIF
Compute Variance Inflation Factors (VIF). VIF > 5–10 can indicate problematic multicollinearity.

In [ ]:
X_train_vif = sm.add_constant(X_train)
vif_df = pd.DataFrame({
    'feature': X_train_vif.columns,
    'VIF': [variance_inflation_factor(X_train_vif.values, i) for i in range(X_train_vif.shape[1])]
})
vif_df

Create an intentionally collinear feature and re-evaluate VIF to see it spike.

In [ ]:
X_train_collinear = X_train.copy()
X_train_collinear['x_dup'] = X_train_collinear['x1'] * 1.0 + np.random.normal(0, 0.01, size=len(X_train_collinear))
X_train_collinear_const = sm.add_constant(X_train_collinear)
vif_collinear = pd.DataFrame({
    'feature': X_train_collinear_const.columns,
    'VIF': [variance_inflation_factor(X_train_collinear_const.values, i) for i in range(X_train_collinear_const.shape[1])]
})
vif_collinear.sort_values('VIF', ascending=False).head(10)

## 4) Optional: Statsmodels OLS summary
Provides coefficients, standard errors, and more detailed diagnostics. Useful for interpretation when assumptions are (approximately) satisfied.

In [ ]:
X_train_const = sm.add_constant(X_train)
ols_train = sm.OLS(y_train, X_train_const).fit()
ols_train.summary()

## 5) Handling nonlinearity and scaling (brief)
Use polynomial features to capture curvature. Note: increases risk of multicollinearity — use scaling and possibly regularization in practice.

In [ ]:
poly_pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('linreg', LinearRegression())
])
poly_pipe.fit(X_train, y_train)
y_pred_poly = poly_pipe.predict(X_test)
r2_poly = r2_score(y_test, y_pred_poly)
rmse_poly = mean_squared_error(y_test, y_pred_poly, squared=False)
print({'R2_linear': r2, 'RMSE_linear': rmse, 'R2_poly': r2_poly, 'RMSE_poly': rmse_poly})

## Exercises
Complete the tasks below. Instructor solution cells are hidden; expand them if needed.

1. Outliers: Inject a few extreme outliers into the training target and re-fit the model. How do metrics and residual plots change? Try trimming or using a robust regression (HuberRegressor) and compare.
2. California Housing: Load `fetch_california_housing()` and repeat the workflow (train/test split, fit, diagnostics). Compare assumptions and performance vs. synthetic data.
3. Feature Engineering: Try adding interaction terms or log-transforming skewed features (on California data). Re-check VIF, residuals, and metrics.


In [ ]:
# Exercise 1: Outliers
# TODO: Copy X_train, y_train to new variables, inject outliers into y, fit LinearRegression, and compare metrics/plots.
...

In [ ]:
# Solution 1 (hidden)
from sklearn.linear_model import HuberRegressor

X_train_o = X_train.copy()
y_train_o = y_train.copy()
idx = np.random.choice(len(y_train_o), size=10, replace=False)
y_train_o.iloc[idx] += np.random.normal(300, 50, size=len(idx))

lr_o = LinearRegression().fit(X_train_o, y_train_o)
y_pred_o = lr_o.predict(X_test)
print('Linear with outliers:', r2_score(y_test, y_pred_o), mean_squared_error(y_test, y_pred_o, squared=False))

hub = HuberRegressor().fit(X_train_o, y_train_o)
y_pred_h = hub.predict(X_test)
print('Huber (robust):', r2_score(y_test, y_pred_h), mean_squared_error(y_test, y_pred_h, squared=False))


In [ ]:
# Exercise 2: California Housing
# TODO: Load dataset, fit LinearRegression, run diagnostics (residual plot, QQ, BP test, VIF), compare to synthetic.
...

In [ ]:
# Solution 2 (hidden)
cal = fetch_california_housing()
Xc = pd.DataFrame(cal.data, columns=cal.feature_names)
yc = pd.Series(cal.target, name='MedHouseVal')
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.2, random_state=42)
lr_c = LinearRegression().fit(Xc_train, yc_train)
yc_pred = lr_c.predict(Xc_test)
print('California R2, RMSE:', r2_score(yc_test, yc_pred), mean_squared_error(yc_test, yc_pred, squared=False))

resid_c = yc_test - yc_pred
fig, axes = plt.subplots(1, 2, figsize=(12,4))
axes[0].scatter(yc_pred, resid_c, alpha=0.5)
axes[0].axhline(0, color='red', ls='--')
axes[0].set_title('Residuals vs Predicted (CA)')
sm.ProbPlot(resid_c, fit=True).qqplot(line='45', ax=axes[1])
axes[1].set_title('QQ Plot (CA)')
plt.show()

Xc_test_const = sm.add_constant(Xc_test)
bp = het_breuschpagan(resid_c, Xc_test_const)
print('BP p-value:', bp[1], bp[3])

Xc_train_const = sm.add_constant(Xc_train)
vif_ca = pd.DataFrame({
    'feature': Xc_train_const.columns,
    'VIF': [variance_inflation_factor(Xc_train_const.values, i) for i in range(Xc_train_const.shape[1])]
})
vif_ca.sort_values('VIF', ascending=False)


In [ ]:
# Exercise 3: Feature Engineering
# TODO: Try interactions or log-transform certain features (e.g., log(1 + MedInc)) and compare metrics/assumptions.
...

In [ ]:
# Solution 3 (hidden)
Xc2 = Xc.copy()
if 'MedInc' in Xc2.columns:
    Xc2['log_MedInc'] = np.log1p(Xc2['MedInc'])
if set(['AveRooms','AveBedrms']).issubset(Xc2.columns):
    Xc2['Rooms_per_Bedroom'] = Xc2['AveRooms'] / (Xc2['AveBedrms'] + 1e-6)

Xc2_train, Xc2_test, yc_train, yc_test = train_test_split(Xc2, yc, test_size=0.2, random_state=42)
lr2 = LinearRegression().fit(Xc2_train, yc_train)
yc2_pred = lr2.predict(Xc2_test)
print('Engineered R2, RMSE:', r2_score(yc_test, yc2_pred), mean_squared_error(yc_test, yc2_pred, squared=False))


## Wrap-up
Checklist for linear regression practice:
- [ ] Split data properly and avoid leakage
- [ ] Fit baseline model and record metrics
- [ ] Plot residuals vs. predicted; look for patterns
- [ ] Check residual normality (hist/QQ), run normality tests if needed
- [ ] Test homoscedasticity (e.g., Breusch–Pagan)
- [ ] Compute VIF for multicollinearity
- [ ] Revisit outliers, transformations, and features as needed
